# Источники информации

#### Проекты НПА

*Ресурс*: https://regulation.gov.ru/ 

*Автоматизация сбора данных*: есть API - https://regulation.gov.ru/opendata/7710349494-npalist/, в паспорте неактуальная ссылка, актуальная – https://regulation.gov.ru/api/npalist;

============================================================================================================

*Ресурс*: https://sozd.duma.gov.ru/

*Автоматизация сбора данных*: есть API - http://api.duma.gov.ru/, но необходимо оформлять доступ; сайт пропускает запросы html-кода страниц.

============================================================================================================

*Ресурс*: https://www.cbr.ru/project_na/

*Автоматизация сбора данных*: ЦБ размещает (почти) все проекты НПА на https://regulation.gov.ru/; сайт пропускает запросы html-кода страниц.



#### НПА

*Ресурс*: http://pravo.gov.ru/ 

*Автоматизация сбора данных*: есть API - http://publication.pravo.gov.ru/help. Возможны запросы по конкретному органу власти.

In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import json
import time  
from bs4 import BeautifulSoup
import datetime as DT
from fake_useragent import UserAgent
UserAgent().chrome
import warnings
warnings.simplefilter('ignore')
pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.set_option('display.max_colwidth', None)

In [2]:
### Название госоргана

FOIV_NAME = 'Министерство цифрового развития, связи и массовых коммуникаций Российской Федерации'
FOIV_SHORT_NAME = 'Минцифры России'

### Ключевое слово

KEY_WORD = 'цифр'

### Временной промежуток сбора данных – при необходимости

START_DATE = '01.04.2023'
END_DATE = '31.05.2023'

# Варианты реализации автоматизированного сбора данных

## https://regulation.gov.ru/

Скрипт запрашивает у API портала regulation.gov.ru информацию по 500 последним проектам НПА и отбирает проекты указанного органа власти или с ключевым словом в тексте названия. Ключевое слово и орган власти указаны в ячейке выше.

In [3]:
def get_xml_data_from_api(api_url):
    response = requests.get(api_url, headers=header)
    return response.content.decode('utf-8')

def decode_xml_to_dataframe(xml_data):
    root = ET.fromstring(xml_data)
    data = []

    for item in root.findall('.//project'):
        row = {}
        for child in item:
            row[child.tag] = child.text
        data.append(row)

    df = pd.DataFrame(data)
    return df

api_url = "https://regulation.gov.ru/api/npalist?limit=500&sort=desc"
header = { 
'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36' 
}

xml_data = get_xml_data_from_api(api_url)
df_regulation = decode_xml_to_dataframe(xml_data)

df_regulation = df_regulation[
    (df_regulation['department'] == FOIV_NAME) |
    (df_regulation['department'] == FOIV_SHORT_NAME) |
    df_regulation['title'].str.contains(KEY_WORD)][
    ['title', 'department', 'projectId', 'kind']].reset_index(drop=True)

def site_regulation(row):
    id_regulation = row['projectId'].split('23/')[-1]
    row['ресурс'] = 'regulation.gov.ru'
    row['ссылка'] = f'https://regulation.gov.ru/Regulation/Npa/PublicView?npaID={id_regulation}'
    return(row)

df_regulation = df_regulation.apply(site_regulation, axis=1)
df_regulation.columns=['нпа',
                       'инициатор',
                       'id нпа в системе',
                       'форма нпа',
                       'ресурс',
                       'ссылка на нпа на ресурсе']
df_regulation

,нпа,инициатор,id нпа в системе,форма нпа,ресурс,ссылка на нпа на ресурсе
0,"Приказ Минцифры России «О раскрытии информации субъектами естественных монополий, осуществляющими деятельность в области оказания услуг связи»",Минцифры России,02/08/05-23/00138598,Проект ведомственного акта,regulation.gov.ru,https://regulation.gov.ru/Regulation/Npa/PublicView?npaID=00138598
1,проект постановления Правительства Российской Федерации «О внесении изменений в Правила предоставления субсидии из федерального бюджета Российскому фонду развития информационных технологий на поддержку проектов по разработке и внедрению российских решений в сфере информационных технологий»,Минцифры России,04/12/05-23/00138597,Проект постановления Правительства Российской Федерации,regulation.gov.ru,https://regulation.gov.ru/Regulation/Npa/PublicView?npaID=00138597
2,проект постановления Правительства Российской Федерации «О внесении изменений в Правила предоставления субсидии из федерального бюджета Российскому фонду развития информационных технологий на поддержку проектов по разработке и внедрению российских решений в сфере информационных технологий»,Минцифры России,04/12/05-23/00138581,Проект постановления Правительства Российской Федерации,regulation.gov.ru,https://regulation.gov.ru/Regulation/Npa/PublicView?npaID=00138581
3,О внесении изменений в некоторые акты Правительства Российской Федерации,Минцифры России,01/01/05-23/00138555,Проект постановления Правительства Российской Федерации,regulation.gov.ru,https://regulation.gov.ru/Regulation/Npa/PublicView?npaID=00138555
4,О внесении изменения в пункт 2 постановления Правительства Российской Федерации от 28 июня 2022 г. № 1152,Минцифры России,01/01/05-23/00138437,Проект постановления Правительства Российской Федерации,regulation.gov.ru,https://regulation.gov.ru/Regulation/Npa/PublicView?npaID=00138437
5,"Приказ ""Об утверждении федерального государственного образовательного \nстандарта среднего профессионального образования по специальности 46.02.ХХ Обеспечение технологического сопровождения цифровой трансформации документированных сфер деятельности""",Минпросвещения России,01/02/05-23/00138334,Проект ведомственного акта,regulation.gov.ru,https://regulation.gov.ru/Regulation/Npa/PublicView?npaID=00138334
6,"О внесении изменений в Порядок представления сведений о базе расчета обязательных отчислений (неналоговых платежей) в резерв универсального обслуживания, утвержденный приказом Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации от 10.11.2021 № 1164",Минцифры России,02/08/05-23/00138252,Проект ведомственного акта,regulation.gov.ru,https://regulation.gov.ru/Regulation/Npa/PublicView?npaID=00138252
7,"О внесении изменений в Порядок составления и утверждения отчетов \nо результатах деятельности федеральных государственных учреждений, подведомственных Минцифры России, и об использовании закрепленного \nза ними государственного имущества, утвержденный приказом Минцифры России от 22 августа 2022 г. № 608",Минцифры России,01/02/05-23/00138186,Проект ведомственного акта,regulation.gov.ru,https://regulation.gov.ru/Regulation/Npa/PublicView?npaID=00138186
8,"О внесении изменений в Правила предоставления субсидий из федерального бюджета российским кредитным организациям на возмещение недополученных ими доходов по кредитам, выданным аккредитованным системообразующим организациям в сфере информационных технологий, а также организациям, входящим в группу лиц системообразующей организации в сфере информационных технологий",Минцифры России,04/12/05-23/00138151,Проект постановления Правительства Российской Федерации,regulation.gov.ru,https://regulation.gov.ru/Regulation/Npa/PublicView?npaID=00138151
9,"Проект постановления Правительства Российской Федерации «О государственной информационной системе «Единый реестр сведений о гражданах, подлежащих первоначальной постановке на воинский учет, гражданах, состоящих на воинском учете, а также о гражданах, не состоящих, но обязанных состоять на воинс

# http://pravo.gov.ru/

Cкрипт запрашивает у API портала pravo.gov.ru информацию по опубликованным НПА, подписанным указанным ФОИВ за текущий месяц

In [4]:
def get_json_byte_data_from_api(url):
    
    response = requests.get(url)
    json_byte_data = response.content
    
    return json_byte_data

def convert_json_byte_to_dataframe(json_byte_data):
    
    json_data = json_byte_data.decode('utf-8')
    data = pd.DataFrame(json.loads(json_data)['items'])
    df = pd.DataFrame(data)

    return df

api_url = 'http://publication.pravo.gov.ru/api/SignatoryAuthorities'
json_byte_data = get_json_byte_data_from_api(api_url)
foiv = pd.read_json(json_byte_data.decode('utf-8'))
foiv_id = list(foiv[foiv['name'] == FOIV_NAME]['id'])

api_url = f'http://publication.pravo.gov.ru/api/Documents?SignatoryAuthorityId={foiv_id[0]}&PeriodType=monthly'
json_byte_data = get_json_byte_data_from_api(api_url)
eonumber_list = list(
    convert_json_byte_to_dataframe(json_byte_data)['eoNumber'])
eonumber_list

df_pravo = pd.DataFrame()
for eonumber in eonumber_list:
    api_url = f'http://publication.pravo.gov.ru/api/Document?eoNumber={eonumber}'
    json_byte_data = get_json_byte_data_from_api(api_url)
    df = pd.DataFrame(list(json.loads(json_byte_data.decode('utf-8')).items())).transpose()
    df = pd.DataFrame(df.values[1:], columns=df.iloc[0])
    df_pravo = pd.concat([df_pravo, df]).reset_index(drop=True)

def site_pravo(row):
    row['форма нпа'] = row['description'].split(' от')[0]
    row['инициатор'] = FOIV_NAME
    row['ресурс'] = 'publication.pravo.gov.ru'
    id_pravo = row['eoNumber']
    row['ссылка на нпа на ресурсе'] = f'http://publication.pravo.gov.ru/document/{id_pravo}'
    return(row)
    
df_pravo = df_pravo[['description', 'eoNumber']].apply(site_pravo, axis=1)
df_pravo.rename(columns={'description':'нпа', 'eoNumber':'id нпа в системе'}, inplace=True)
df_pravo

,нпа,id нпа в системе,форма нпа,инициатор,ресурс,ссылка на нпа на ресурсе
0,"Приказ Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации, Федеральной службы безопасности Российской Федерации от 29.03.2023 № 321/147\n ""Об утверждении Типовых требований к плану мероприятий по внедрению собственниками или иными владельцами технологических сетей связи, имеющих уникальный идентификатор совокупности средств связи и иных технических средств в информационно- телекоммуникационной сети ""Интернет"", технических средств, обеспечивающих выполнение установленных действий при проведении оперативно-разыскных мероприятий""",0001202305250029,"Приказ Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации, Федеральной службы безопасности Российской Федерации","Министерство цифрового развития, связи и массовых коммуникаций Российской Федерации",publication.pravo.gov.ru,http://publication.pravo.gov.ru/document/0001202305250029
1,"Приказ Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации от 20.03.2023 № 246\n ""О внесении изменений в приказ Министерства связи и массовых коммуникаций Российской Федерации от 19 января 2016 г. № 3 ""Об утверждении Требований к порядку организационно-технического взаимодействия операторов подвижной радиотелефонной связи при обеспечении перенесения абонентского номера""",0001202305250026,"Приказ Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации","Министерство цифрового развития, связи и массовых коммуникаций Российской Федерации",publication.pravo.gov.ru,http://publication.pravo.gov.ru/document/0001202305250026
2,"Приказ Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации от 17.04.2023 № 378\n ""Об утверждении методик проверки соответствия предоставленных биометрических персональных данных физического лица соответствующим векторам единой биометрической системы и определении степени взаимного соответствия биометрических персональных данных и векторов единой биометрической системы, достаточной для проведения идентификации и (или) аутентификации""",0001202305240002,"Приказ Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации","Министерство цифрового развития, связи и массовых коммуникаций Российской Федерации",publication.pravo.gov.ru,http://publication.pravo.gov.ru/document/0001202305240002
3,"Приказ Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации от 09.11.2022 № 841\n ""Об утверждении положений о функциональных подсистемах информационно-технологической инфраструктуры и электросвязи и почтовой связи единой государственной системы предупреждения и ликвидации чрезвычайных ситуаций""",0001202305220030,"Приказ Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации","Министерство цифрового развития, связи и массовых коммуникаций Российской Федерации",publication.pravo.gov.ru,http://publication.pravo.gov.ru/document/0001202305220030
4,"Приказ Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации от 20.04.2023 № 387\n ""Об утверждении требований к деловой репутации единоличного исполнительного органа, а также членов коллегиального исполнительного органа и (или) физических лиц - учредителей (участников), имеющих право распоряжаться более 10 процентами акций (долей), составляющих уставный капитал организации, осуществляющей аутентификацию на основе биометрических персональных данных физических лиц, и о признании утратившим силу приказа Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации от 27 августа 2021 г. № 896""",0001202305190014,"Приказ Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации","Министерство цифрового развития, связи и массовых коммуникаций Российской Федерации",publication.pravo.gov.ru,http://publication.pravo.gov.ru/document/0001202305190014
5,"Приказ Министерства цифро

## https://sozd.duma.gov.ru

Cкрипт собирает со страницы поиска по законопроектам список ссылок на проекты, где упоминается ключевое слово, с мероприятиями в указанные временные промежутки (в рамках тестового задания 01.04.2023-31.05.2023). Со страниц по ссылкам собирается информация по проектам НПА, подходящим под критерии.

В реальной задаче для упрощения запросов будет необходимо получить доступ к API.


In [5]:
links = [f'https://sozd.duma.gov.ru/oz?b[Annotation]={KEY_WORD}&date_period_from_ExistsEventsDate={START_DATE}&date_period_to_ExistsEventsDate={END_DATE}&b[ExistsEventsDate]={START_DATE}%20-%20{END_DATE}&b[ClassOfTheObjectLawmakingId]=1&#data_source_tab_b']

link_list = []

for link in links:
    response = requests.get(link, headers={'User-Agent': UserAgent().chrome}, verify=False)
    html_duma = response.content
    soup_duma = BeautifulSoup(html_duma,'html.parser')
    obj_duma = soup_duma.findAll('a', class_='lnk o_num')
    for obj in obj_duma:
        href = obj.get('href')
        link_list.append(href)

df_sozd = pd.DataFrame([])        
        
for link in link_list:

    response = requests.get(f'https://sozd.duma.gov.ru{link}', headers={'User-Agent': UserAgent().chrome}, verify=False)
    html_duma = response.content
    soup_duma = BeautifulSoup(html_duma,'html.parser')

    z_type = soup_duma.findAll(
        'div', class_='opch_r'
    )[1].text.strip()

    oz_name = soup_duma.find(
        'span', class_='oz_naimen', id="oz_name"
    ).text.strip()

    initiator = soup_duma.findAll(
        'div', class_='opch_r'
    )[0].text.strip()


    df = pd.DataFrame(
        [
        'sozd.duma.gov.ru',
        z_type,
        oz_name,
        initiator,
        f'https://sozd.duma.gov.ru{link}',
        link
    ]
    ).transpose()

    df_sozd = df_sozd.append(df)
    
df_sozd.columns = [
    'ресурс',
    'форма нпа',
    'нпа',
    'инициатор',
    'ссылка на нпа на ресурсе',
    'id нпа в системе']

df_sozd

,ресурс,форма нпа,нпа,инициатор,ссылка на нпа на ресурсе,id нпа в системе
0,sozd.duma.gov.ru,Федеральный закон,"О внесении изменений в отдельные законодательные акты Российской Федерации в связи с принятием Федерального закона ""Об экспериментальных правовых режимах в сфере цифровых инноваций в Российской Федерации"" (в части разработки, апробации и внедрения цифровых инноваций на финансовом рынке)","Депутаты Государственной Думы А.Г.Аксаков, А.Н.Свистунов, И.Н.Бабич, С.В.Алтухов, А.В.Горелкин",https://sozd.duma.gov.ru/bill/341257-8,/bill/341257-8
0,sozd.duma.gov.ru,Федеральный закон,"О внесении изменений в Федеральный закон ""О цифровых финансовых активах, цифровой валюте и о внесении изменений в отдельные законодательные акты Российской Федерации""","Депутаты Государственной Думы А.Г.Аксаков, А.В.Гордеев, В.А.Даванков, Я.Е.Нилов, А.Н.Свистунов, И.Н.Бабич, В.Б.Сенин, О.Д.Димов, В.С.Макаров, А.В.Горелкин, А.К.Луговой, А.О.Ткачев, В.В.Плякин, С.В.Авксентьева, С.В.Алтухов, Д.Н.Гасанов",https://sozd.duma.gov.ru/bill/237585-8,/bill/237585-8
0,sozd.duma.gov.ru,Федеральный закон,О внесении изменений в отдельные законодательные акты Российской Федерации в связи с внедрением цифрового рубля,"Депутаты Государственной Думы А.Г.Аксаков, К.М.Бахарев, А.Н.Свистунов, И.Н.Бабич, О.Д.Димов, В.С.Макаров, С.В.Алтухов; Сенаторы Российской Федерации Н.А.Журавлев, Г.Н.Карелова, А.Д.Артамонов, М.М.Ульбашев, А.А.Клишас",https://sozd.duma.gov.ru/bill/270838-8,/bill/270838-8


## https://www.cbr.ru/project_na/

Скрипт собирает информацию о проектах НПА и ссылки на документы. Также собирает ссылки на пояснительные записки - при необходимости можно обавить в итоговые таблицы.

Для тестовой задачи были собраны все проекты НПА ЦБ.

In [6]:
response = requests.get('https://www.cbr.ru/project_na/', headers={'User-Agent': UserAgent().chrome}, verify=False)
html_cbr = response.content
soup_cbr = BeautifulSoup(html_cbr,'html.parser')

hrefs_cbr = soup_cbr.findAll(
        'a', target="_blank"
    )[1:-1]

projects_cbr = soup_cbr.findAll(
        'h4'
    )

list_href_projects = []
list_href_notes = []
list_projects = []

for i in range(0, len(hrefs_cbr), 2):
    list_href_projects.append(hrefs_cbr[i].get('href'))
    list_href_notes.append(hrefs_cbr[i+1].get('href'))

for i in range(len(projects_cbr)):
    list_projects.append(projects_cbr[i].text.strip())

df_cbr = pd.DataFrame([list_projects, list_href_projects]).transpose()
df_cbr.columns=['нпа', 'ссылка на нпа на ресурсе']

def site_cbr(row):
    href = row['ссылка на нпа на ресурсе']
    row['ссылка на нпа на ресурсе'] = f'https://www.cbr.ru{href}'
    row['форма нпа'] = row['нпа'].split(' «')[0]
    row['инициатор'] = 'Банк России'
    row['ресурс'] = 'https://www.cbr.ru/project_na/'
    row['id нпа в системе'] = row['ссылка на нпа на ресурсе'][-4:]
    return(row)

df_cbr = df_cbr.apply(site_cbr, axis=1)
df_cbr

,нпа,ссылка на нпа на ресурсе,форма нпа,инициатор,ресурс,id нпа в системе
0,Проект указания Банка России «О внесении изменений в Указание Банка России от 5 октября 2021 года № 5969-У»,https://www.cbr.ru/Queries/XsltBlock/File/90538/4993,Проект указания Банка России,Банк России,https://www.cbr.ru/project_na/,4993
1,"Проект Указания Банка России «О внесении изменений в Положение Банка России от 16 ноября 2021 года № 780-П «О временной администрации негосударственного пенсионного фонда, назначаемой Банком России в связи с аннулированием лицензий на осуществление деятельности по пенсионному обеспечению и пенсионному страхованию»",https://www.cbr.ru/Queries/XsltBlock/File/90538/4992,Проект Указания Банка России,Банк России,https://www.cbr.ru/project_na/,4992
2,"Проект указания Банка России «О внесении изменений в Положение Банка России от 30.12.2016 № 575-П «О требованиях к управлению рисками, правилам организации системы управления рисками, клиринговому обеспечению, размещению имущества, формированию активов центрального контрагента, а также к кругу лиц, в которых центральный контрагент имеет право открывать торговые и клиринговые счета, и методике определения выделенного капитала центрального контрагента»",https://www.cbr.ru/Queries/XsltBlock/File/90538/4991,Проект указания Банка России,Банк России,https://www.cbr.ru/project_na/,4991
3,"Проект указания Банка России «О внесении изменений в Указание Банка России от 12 сентября 2018 года № 4905-У «О требованиях к деятельности центрального депозитария в части организации управления рисками, связанными с осуществлением деятельности центрального депозитария, а также к правилам управления рисками, связанными с осуществлением деятельности центрального депозитария»",https://www.cbr.ru/Queries/XsltBlock/File/90538/4990,Проект указания Банка России,Банк России,https://www.cbr.ru/project_na/,4990
4,"Проект указания Банка России «О внесении изменений в Указание Банка России от 29.09.2016 № 4144-У «О требованиях к системе управления рисками, связанными с осуществлением репозитарной деятельности, и правилам управления рисками репозитария»",https://www.cbr.ru/Queries/XsltBlock/File/90538/4989,Проект указания Банка России,Банк России,https://www.cbr.ru/project_na/,4989
5,"Проект указания Банка России «О внесении изменений в Указание Банка России от 07.05.2018 № 4791-У «О требованиях к организации организатором торговли системы управления рисками, связанными с организацией торгов, а также с осуществлением операций с собственным имуществом, и к документам организатора торговли, определяющим меры, направленные на снижение указанных рисков и предотвращение конфликта интересов»",https://www.cbr.ru/Queries/XsltBlock/File/90538/4988,Проект указания Банка России,Банк России,https://www.cbr.ru/project_na/,4988
6,"Проект указания Банка России «О внесении изменений в Положение Банка России от 12.03.2015 № 463-П «О требованиях, направленных на снижение рисков осуществления клиринговой деятельности, и требованиях к документу (документам), определяющему (определяющим) меры, направленные на снижение кредитных, операционных и иных рисков, в том числе рисков, связанных с совмещением клиринговой деятельности с иными видами деятельности»",https://www.cbr.ru/Queries/XsltBlock/File/90538/4987,Проект указания Банка России,Банк России,https://www.cbr.ru/project_na/,4987
7,"Проект положения Банка России «О требованиях к управлению операционным риском организаторов торговли, клиринговых организаций, центральных контрагентов, центрального депозитария и репозитариев»",https://www.cbr.ru/Queries/XsltBlock/File/90538/4986,Проект положения Банка России,Банк России,https://www.cbr.ru/project_na/,4986
8,Проект указания Банка России «О внесении изменений в Положение Банка России от 16 ноября 2021 года № 781-П «О требованиях к финансовой устойчивости и платежеспособности страховщиков»,https://www.cbr.ru/Queries/XsltBlock/File/90538/4985,Проект указания Банка России,Банк России,https://www.cbr.ru/project_na/,4985
9,Проект указания

# Собираем все в одну базу НПА и проектов НПА

In [7]:
npa_base = df_sozd.append(df_regulation).append(df_pravo).append(df_cbr).reset_index(drop=True)
npa_base

,ресурс,форма нпа,нпа,инициатор,ссылка на нпа на ресурсе,id нпа в системе
0,sozd.duma.gov.ru,Федеральный закон,"О внесении изменений в отдельные законодательные акты Российской Федерации в связи с принятием Федерального закона ""Об экспериментальных правовых режимах в сфере цифровых инноваций в Российской Федерации"" (в части разработки, апробации и внедрения цифровых инноваций на финансовом рынке)","Депутаты Государственной Думы А.Г.Аксаков, А.Н.Свистунов, И.Н.Бабич, С.В.Алтухов, А.В.Горелкин",https://sozd.duma.gov.ru/bill/341257-8,/bill/341257-8
1,sozd.duma.gov.ru,Федеральный закон,"О внесении изменений в Федеральный закон ""О цифровых финансовых активах, цифровой валюте и о внесении изменений в отдельные законодательные акты Российской Федерации""","Депутаты Государственной Думы А.Г.Аксаков, А.В.Гордеев, В.А.Даванков, Я.Е.Нилов, А.Н.Свистунов, И.Н.Бабич, В.Б.Сенин, О.Д.Димов, В.С.Макаров, А.В.Горелкин, А.К.Луговой, А.О.Ткачев, В.В.Плякин, С.В.Авксентьева, С.В.Алтухов, Д.Н.Гасанов",https://sozd.duma.gov.ru/bill/237585-8,/bill/237585-8
2,sozd.duma.gov.ru,Федеральный закон,О внесении изменений в отдельные законодательные акты Российской Федерации в связи с внедрением цифрового рубля,"Депутаты Государственной Думы А.Г.Аксаков, К.М.Бахарев, А.Н.Свистунов, И.Н.Бабич, О.Д.Димов, В.С.Макаров, С.В.Алтухов; Сенаторы Российской Федерации Н.А.Журавлев, Г.Н.Карелова, А.Д.Артамонов, М.М.Ульбашев, А.А.Клишас",https://sozd.duma.gov.ru/bill/270838-8,/bill/270838-8
3,regulation.gov.ru,Проект ведомственного акта,"Приказ Минцифры России «О раскрытии информации субъектами естественных монополий, осуществляющими деятельность в области оказания услуг связи»",Минцифры России,https://regulation.gov.ru/Regulation/Npa/PublicView?npaID=00138598,02/08/05-23/00138598
4,regulation.gov.ru,Проект постановления Правительства Российской Федерации,проект постановления Правительства Российской Федерации «О внесении изменений в Правила предоставления субсидии из федерального бюджета Российскому фонду развития информационных технологий на поддержку проектов по разработке и внедрению российских решений в сфере информационных технологий»,Минцифры России,https://regulation.gov.ru/Regulation/Npa/PublicView?npaID=00138597,04/12/05-23/00138597
5,regulation.gov.ru,Проект постановления Правительства Российской Федерации,проект постановления Правительства Российской Федерации «О внесении изменений в Правила предоставления субсидии из федерального бюджета Российскому фонду развития информационных технологий на поддержку проектов по разработке и внедрению российских решений в сфере информационных технологий»,Минцифры России,https://regulation.gov.ru/Regulation/Npa/PublicView?npaID=00138581,04/12/05-23/00138581
6,regulation.gov.ru,Проект постановления Правительства Российской Федерации,О внесении изменений в некоторые акты Правительства Российской Федерации,Минцифры России,https://regulation.gov.ru/Regulation/Npa/PublicView?npaID=00138555,01/01/05-23/00138555
7,regulation.gov.ru,Проект постановления Правительства Российской Федерации,О внесении изменения в пункт 2 постановления Правительства Российской Федерации от 28 июня 2022 г. № 1152,Минцифры России,https://regulation.gov.ru/Regulation/Npa/PublicView?npaID=00138437,01/01/05-23/00138437
8,regulation.gov.ru,Проект ведомственного акта,"Приказ ""Об утверждении федерального государственного образовательного \nстандарта среднего профессионального образования по специальности 46.02.ХХ Обеспечение технологического сопровождения цифровой трансформации документированных сфер деятельности""",Минпросвещения России,https://regulation.gov.ru/Regulation/Npa/PublicView?npaID=00138334,01/02/05-23/00138334
9,regulation.gov.ru,Проект ведомственного акта,"О внесении изменений в Порядок представления сведений о базе расчета обязательных отчислений (неналоговых платежей) в резерв универсального обслуживания, утвержденный приказом Министерства цифрового развития, связи и массовых коммуникаций Российской Федерации от 10.11.2021 № 1164",Минцифр

# Дальнейшая оптимизация процесса

* Автоматизация уведомлений ответственных сотрудников по ключевым словам в названии нпа/проектов, инициаторам;

*Реализация* – совместно с сотрудниками, отвечающими за тикеты


* Мониторинг изменений информации по законопроектам\инициативам, автоматизированные уведомления ответственных сотрудников о ходе проектов в сфере их ответственности;

*Реализация* – сравнение с ранее полученными данными или с последней датой таймлайна проекта НПА: при наличии изменений обновление данных и проставление маркера изменений


* Автоматизированное распределение карточек проектов НПА и НПА по ответственным на основе содержания текстов;

*Реализация* – нейронная сеть, обученная на ответах аналитиков/экспертов о релевантности проекта НПА и НПА: высокая сложность, необходимо привлечь специалистов по Data Science

### Вариант реализации процесса

Периодический автоматизированный запрос данных с ресурсов фиксирует размещение инциативы или проекта НПА и собирает о нем данные. Определяется сфера проекта по названию и инициатору, другим маркерам (в перспективе – по тексту проекта) и ответственные сотрудники.

==========

Данные сравниваются с предыдущей базой на наличие дубликатов. Если дубликата нет по id или другому параметру нет, сотрудника информируют о первичном размещении проекта НПА.

==========

Если проект по id или другому параметру определяется как дубликат, но не полный, или ресурс предполагает таймлайн по проекту НПА, сотрудника информируют о наличии изменений по проекту.

==========

Публикация на http://pravo.gov.ru является последней точкой, когда сотрудника информируют о движении по проекту, отслеживание заканчивается.